## Import packages

In [ ]:
# !pip install -q matplotlib
# !pip install -q plotly
# !pip install -q ydata-profiling
# !pip install -q -U nbformat
# !pip install -q --upgrade pip setuptools
# !pip install -q --force-reinstall nbformat
!pip install -q prophet

In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pickle


import warnings
warnings.filterwarnings("ignore")

In [ ]:
# load data
df = pd.read_csv("../data/PJME_hourly.csv")
df.head()

In [ ]:
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis=0, ascending=True, inplace=True)
df.reset_index(inplace=True, drop=True)

df.rename(columns={'PJME_MW':'demand_in_MW'}, inplace=True)

df.head()

In [ ]:
df.shape

In [ ]:
print("duplicates -", df.duplicated(subset="Datetime").sum())

In [ ]:
df.drop_duplicates(subset='Datetime', keep='last', inplace=True)
print(df.shape)

In [ ]:
# let's see if we have a continuous dataset
df = df.set_index('Datetime')
print(f'df.index.freq is set to: {df.index.freq}')

# The fact our datetime index's frequency is set to None is an indication there are some missing data points
# somewhere (otherwise Python could deduce it). Let's compare it to an uninterruped custom date range.

In [ ]:
date_range = pd.date_range(start=min(df.index),
                           end=max(df.index),
                           freq='H')

In [ ]:
print(f'The difference in length between the custom date range and our dataset is {(len(date_range)-len(df))}:')
print(date_range.difference(df.index))

In [ ]:
# this will append the previously missing datetimes, and create null values in our target variable
df = df.reindex(date_range)

# we fill in the blanks with values that lie on a linear curve between existing data points
df['demand_in_MW'].interpolate(method='linear', inplace=True)

# now we have a neatly continuous datetime index
print(f'The df.index.freq is now: {df.index.freq}, indicating that we no longer have missing instances')

## datetime features

In [ ]:
df['dow'] = df.index.dayofweek
df['doy'] = df.index.dayofyear
df['year'] = df.index.year
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['hour'] = df.index.hour
df['weekday'] = df.index.weekday
df['woy'] = df.index.isocalendar().week
df['dom'] = df.index.day
df['date'] = df.index.date

# let's add the season number
df['season'] = df['month'].apply(lambda month_number: (month_number % 12 + 3)//3)

In [ ]:
df.head()

In [ ]:
df.dtypes

## Plotting

In [ ]:
df['date_and_time'] = df.index

# plotting
fig = px.line(df,
              x='date_and_time',
              y='demand_in_MW',
              title=f'Power Demand (MW) over time [{min(df.year)} - {max(df.year)}]')
fig.update_traces(line=dict(width=0.05))
fig.update_layout(xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

In [ ]:
# power demand throughout the day for each weekday

# aggregated data
grouped_data = (
    df
    .groupby(['hour', 'weekday'], as_index=False)
    .agg({'demand_in_MW':'median'})
)

# plotting
fig = px.line(grouped_data,
              x='hour',
              y='demand_in_MW',
              color='weekday',
              title='Median Hourly Power Demand per Weekday')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='Energy Demand [MW]')
fig.show()

In [ ]:
# aggregated data
grouped_data2 = (
    df
    .groupby(['hour', 'season'], as_index=False)\
    .agg({'demand_in_MW':'median'})
)

# plotting
fig = px.line(grouped_data2,
              x='hour',
              y='demand_in_MW',
              color='season',
              title='Median Hourly Power Demand per Season')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='Energy Demand [MW]')
fig.show()

## Time series decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# seasonal_decompose needs a dataframe with a datetime index
series = df[['demand_in_MW']]
frequency = 24*365

# decomposing the time-series, with the frequency being 24 hours per 365 days
decomposed = seasonal_decompose(series, model='additive',period=frequency)

In [ ]:
# draw a line plot of the data
fig = px.line(decomposed.trend,
        y="trend",
        title="Trend",
        height=300)

# adjust line width
fig.update_traces(line=dict(width=2))

# change layout of axes and the figure's margins
# to emulate tight_layout
fig.update_layout(
        xaxis=dict(
        showticklabels=False,
        linewidth=1
        ),
        yaxis=dict(title=''),
        margin=go.layout.Margin(
            l=40, r=40, b=0, t=40, pad=0
        ),
    )

# display
fig.show()

In [ ]:
fig = px.line(decomposed.seasonal,
        y="seasonal",
        title="Seasonality",
        height=300)

fig.update_traces(line=dict(width=0.025))

fig.update_layout(
        xaxis=dict(
        showticklabels=False,
        linewidth=1
        ),
        yaxis=dict(title=''),
        margin=go.layout.Margin(
            l=40, r=40, b=0, t=40, pad=0
        ),
    )

fig.show()

In [ ]:
fig = px.line(decomposed.resid,
        y="resid",
        title="Residuals",
        height=300)

fig.update_traces(line=dict(width=0.5))

fig.update_layout(
        xaxis=dict(
        showticklabels=False,
        linewidth=1
        ),
        yaxis=dict(title=''),
        margin=go.layout.Margin(
            l=40, r=40, b=0, t=40, pad=0
        ),
    )

fig.show()

# Forecasting models

## Train test split

In [ ]:
print(f'The last date time point in our dataframe is: {max(df.index)}')

In [ ]:
# set cutoff date manually
CUTOFF_DATE = pd.to_datetime('2017-08-01')
TIME_DELTA = pd.DateOffset(years=8)

# splitting
train = df.loc[(df.index < CUTOFF_DATE) & (df.index >= CUTOFF_DATE-TIME_DELTA) ].copy()
test = df.loc[df.index >= CUTOFF_DATE].copy()

In [ ]:
print(f'Training shape: {train.shape} \nTesting shape: {test.shape}\n')
print(f'The training set lies between the dates: {min(train.index)} and {max(train.index)}')
print(f'For the testing set, the dates are: {min(test.index)} and {max(test.index)}')


## Holt Winters - Triple exponential smoothning

In [ ]:
import statsmodels.api as sm

In [ ]:
# exponential smoothing only takes into consideration patterns in the target variable
# so we discard the other features
exp_smooth_train, exp_smooth_test = train['demand_in_MW'], test['demand_in_MW']

# fit & predict
holt_winter = sm.tsa.ExponentialSmoothing(exp_smooth_train,
                                          seasonal_periods=24*365,
                                          seasonal='add').fit()
y_hat_holt_winter = holt_winter.forecast(len(exp_smooth_test))

In [ ]:
model_filename = 'holt_winter_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(holt_winter, model_file)

In [ ]:
exp_smooth_train, exp_smooth_test = train['demand_in_MW'], test['demand_in_MW']

# To load the model later
with open("holt_winter_model.pkl", 'rb') as model_file:
    loaded_model = pickle.load(model_file)

y_hat_holt_winter = loaded_model.forecast(len(exp_smooth_test))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=exp_smooth_test.index, y=exp_smooth_test,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=y_hat_holt_winter.index, y=y_hat_holt_winter,
                         mode='lines',
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Holt-Winter Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')

In [ ]:
def mape(y_true, y_pred):
    """ Mean Absolute Percentage Error """

    # convert to numpy arrays
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    # take the percentage error
    pe = (y_true - y_pred) / y_true

    # take the absolute values
    ape = np.abs(pe)

    # quantify the performance in a single number
    mape = np.mean(ape)

    return f'{mape*100:.2f}%'

In [ ]:
mape_hw = mape(y_true=exp_smooth_test, y_pred=y_hat_holt_winter)
print(f'Our Holt-Winter model has a mean average percentage error of {mape_hw}')

In [ ]:
# inter day predictions

# interval length
interval = 24 * 7

# intermediary variables for readability
x_true, y_true = exp_smooth_test.iloc[:interval].index, exp_smooth_test.iloc[:interval]
x_pred, y_pred = y_hat_holt_winter.iloc[:interval].index, y_hat_holt_winter.iloc[:interval]

# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode='lines',
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Holt-Winter Intra-Day Forecast of First {interval} Hours of Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for interval of the first {interval} hours: {mape(y_true, y_pred)}')

In [ ]:
# interval length
interval = -24 * 7

# intermediary variables for readability
x_true, y_true = exp_smooth_test.iloc[interval:].index, exp_smooth_test.iloc[interval:]
x_pred, y_pred = y_hat_holt_winter.iloc[interval:].index, y_hat_holt_winter.iloc[interval:]

# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode='lines',
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Holt-Winter Intra-Day Forecast of Last {abs(interval)} Hours of Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for interval of the last {abs(interval)} hours: {mape(y_true, y_pred)}')

In [ ]:
#we use tra.diff()(differenced data), because this time series is unit root process.
# ACF and PACF - partial auto correlation
fig,ax = plt.subplots(2,1,figsize=(20,10))
fig = sm.graphics.tsa.plot_acf( train['demand_in_MW'].diff().dropna(), lags=72, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(train['demand_in_MW'].diff().dropna(), lags=72, ax=ax[1])
plt.show()

## Prophet

In [ ]:
prophet_train = train.reset_index(drop=True)
prophet_train.columns

In [ ]:
prophet_train.rename({"date_and_time":"ds","demand_in_MW":"y"},axis=1,inplace=True)
prophet_train.columns

In [ ]:
# comment this column for multivariate time series analysis

prophet_train.drop(
    columns=["dow","doy","year","month","quarter","hour","weekday","woy","dom","date","season"],
    axis=1,
    inplace=True,
)

In [ ]:
prophet_test = test.reset_index(drop=True)
prophet_test.columns

In [ ]:
prophet_test.rename({"date_and_time":"ds","demand_in_MW":"y"},axis=1,inplace=True)
prophet_test.columns

In [ ]:
prophet_test.drop(
    columns=["dow","doy","year","month","quarter","hour","weekday","woy","dom","date","season"],
    axis=1, inplace=True
)

In [ ]:
# prophet model training

# Importing Prophet
from prophet import Prophet


# Instantiate the Prophet model
prophet_model = Prophet()

# Fit the model to the training data
prophet_model.fit(prophet_train)

# Make future predictions
future = prophet_model.make_future_dataframe(periods=len(prophet_test), freq='H')
forecast = prophet_model.predict(future)

# Extracting predictions for the test period
y_hat_prophet = forecast[-len(prophet_test):]

In [ ]:
y_hat_prophet.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=prophet_test["ds"], y=prophet_test["y"],
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=y_hat_prophet["ds"], y=y_hat_prophet["yhat"],
                         mode='lines',
                         name='Test - Prediction'))

fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')

In [ ]:
model_filename = 'prophet_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(prophet_model, model_file)